# Lesson2 表格型方法—— Q-learning
## 1. Q-learning简介
* `Q-learning`也是采用`Q`表格的方式存储`Q`值（状态动作价值），决策部分与`Sarsa`是一样的，采用`ε-greedy`方式增加探索。
* `Q-learning`跟`Sarsa`不一样的地方是更新`Q`表格的方式。
    * `Sarsa`是`on-policy`的更新方式，先做出动作再更新。
    * `Q-learning`是`off-policy`的更新方式，更新`learn()`时无需获取下一步实际做出的动作`next_action`，并假设下一步动作是取最大`Q`值的动作。
* `Q-learning`的更新公式为：

![](https://ai-studio-static-online.cdn.bcebos.com/38158582039041edad0a5a704ba792d0e464f2eb8a394577bf88051cc52d6b66)

## 2. Q-learning实战
* 使用`Q-learning`解决悬崖问题，找到绕过悬崖通往终端的最短路径。

### Step1 安装依赖


In [3]:
!pip install gym

### Step2 导入依赖

In [4]:
import gym
import time
import numpy as np 

### Step3 Agent
* `Agent`是和环境`environment`交互的主体。
* `predict()`方法：输入观察值`observation`（或者说状态`state`），输出动作值
* `sample()`方法：在`predict()`方法基础上使用`ε-greedy`增加探索
* `learn()`方法：输入训练数据，完成一轮`Q`表格的更新

In [5]:
class QLearningAgent(object):
    def __init__(self, obs_n, act_n, learning_rate=0.01, gamma=0.9, e_greed=0.1):
        self.act_n = act_n      # 动作维度，有几个动作可选
        self.lr = learning_rate # 学习率
        self.gamma = gamma      # reward的衰减率
        self.epsilon = e_greed  # 按一定概率随机选动作
        self.Q = np.zeros((obs_n, act_n))

    # 根据输入观察值，采样输出的动作值，带探索
    def sample(self, obs):
        if np.random.uniform(0, 1) < (1.0 - self.epsilon): #根据table的Q值选动作
            action = self.predict(obs)
        else:
            action = np.random.choice(self.act_n) #有一定概率随机探索选取一个动作
        return action

    # 根据输入观察值，预测输出的动作值
    def predict(self, obs):
        Q_list = self.Q[obs, :]
        maxQ = np.max(Q_list)
        action_list = np.where(Q_list == maxQ)[0]  # maxQ可能对应多个action
        action = np.random.choice(action_list)
        return action

    # 学习方法，也就是更新Q-table的方法
    def learn(self, obs, action, reward, next_obs, done):
        """ off-policy
            obs: 交互前的obs, s_t
            action: 本次交互选择的action, a_t
            reward: 本次动作获得的奖励r
            next_obs: 本次交互后的obs, s_t+1
            done: episode是否结束
        """
        predict_Q = self.Q[obs, action]
        if done:
            target_Q = reward # 没有下一个状态了
        else:
            target_Q = reward + self.gamma * np.max(self.Q[next_obs, :]) # Q-learning
        self.Q[obs, action] += self.lr * (target_Q - predict_Q) # 修正q

    # 把 Q表格 的数据保存到文件中
    def save(self):
        npy_file = './q_table.npy'
        np.save(npy_file, self.Q)
        print(npy_file + ' saved.')
    
    # 从文件中读取数据到 Q表格
    def restore(self, npy_file='./q_table.npy'):
        self.Q = np.load(npy_file)
        print(npy_file + ' loaded.')


### Step4 Training && Test（训练&&测试）
* `run_episode()`：`agent`在一个`episode`中训练的过程，使用`agent.sample()`与环境交互，使用`agent.learn()`训练`Q`表格。
* `test_episode()`：`agent`在一个`episode`中测试效果，评估目前的`agent`能在一个`episode`中拿到多少总`reward`。

In [9]:
# train.py

def run_episode(env, agent, render=False):
    total_steps = 0 # 记录每个episode走了多少step
    total_reward = 0

    obs = env.reset() # 重置环境, 重新开一局（即开始新的一个episode）

    while True:
        action = agent.sample(obs) # 根据算法选择一个动作
        next_obs, reward, done, _ = env.step(action) # 与环境进行一个交互
        # 训练 Q-learning算法
        agent.learn(obs, action, reward, next_obs, done)

        obs = next_obs  # 存储上一个观察值
        total_reward += reward
        total_steps += 1 # 计算step数
        if render:
            env.render() #渲染新的一帧图形
        if done:
            break
    return total_reward, total_steps

def test_episode(env, agent):
    total_reward = 0
    obs = env.reset()
    while True:
        action = agent.predict(obs) # greedy
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        obs = next_obs
        # time.sleep(0.5)
        # env.render()
        if done:
            break
    return total_reward


### Step5 创建环境和Agent，启动训练

In [10]:
# 使用gym创建悬崖环境
env = gym.make("CliffWalking-v0")  # 0 up, 1 right, 2 down, 3 left

# 创建一个agent实例，输入超参数
agent = QLearningAgent(
    obs_n=env.observation_space.n,
    act_n=env.action_space.n,
    learning_rate=0.1,
    gamma=0.9,
    e_greed=0.1)


# 训练500个episode，打印每个episode的分数
for episode in range(500):
    ep_reward, ep_steps = run_episode(env, agent, False)
    print('Episode %s: steps = %s , reward = %.1f' % (episode, ep_steps, ep_reward))

# 全部训练结束，查看算法效果
test_reward = test_episode(env, agent)
print('test reward = %.1f' % (test_reward))

Episode 0: steps = 262 , reward = -955.0
Episode 1: steps = 686 , reward = -1577.0
Episode 2: steps = 381 , reward = -777.0
Episode 3: steps = 352 , reward = -748.0
Episode 4: steps = 201 , reward = -201.0
Episode 5: steps = 279 , reward = -576.0
Episode 6: steps = 241 , reward = -439.0
Episode 7: steps = 72 , reward = -72.0
Episode 8: steps = 149 , reward = -347.0
Episode 9: steps = 176 , reward = -275.0
Episode 10: steps = 119 , reward = -119.0
Episode 11: steps = 276 , reward = -672.0
Episode 12: steps = 186 , reward = -285.0
Episode 13: steps = 90 , reward = -288.0
Episode 14: steps = 58 , reward = -58.0
Episode 15: steps = 135 , reward = -135.0
Episode 16: steps = 83 , reward = -182.0
Episode 17: steps = 184 , reward = -580.0
Episode 18: steps = 50 , reward = -50.0
Episode 19: steps = 114 , reward = -114.0
Episode 20: steps = 57 , reward = -156.0
Episode 21: steps = 37 , reward = -37.0
Episode 22: steps = 55 , reward = -55.0
Episode 23: steps = 162 , reward = -261.0
Episode 24: st